In [1]:
from datamodule.preprocessing import *
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import polars as pl

columns = [  
    (
        pl.col("full_text").str.split(by="\n\n").alias("paragraph")
    ),
]

train = pl.read_csv('data/train.csv').with_columns(columns)[:100]
train_features = pd.read_csv('data/train_features.csv')
train.shape

(100, 4)

In [4]:
tmp = Paragraph_Preprocess(train)
train_feats = Paragraph_Eng(tmp)
train_feats['score'] = train['score']
# Obtain feature names
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

NameError: name 'dataPreprocessing' is not defined

In [42]:
tmp.head()

essay_id,full_text,score,paragraph
str,str,i64,str
"""000d118""","""Many people have car where the…",3,"""many people have car where the…"
"""000fe60""","""I am a scientist at NASA that …",3,"""i am a scientist at nasa that …"
"""001ab80""","""People always wish they had th…",4,"""people always wish they had th…"
"""001bdc0""","""We all heard about Venus, the …",4,"""we all heard about venus, the …"
"""002ba53""","""Dear, State Senator This is a…",3,"""dear, state senator this is a …"


In [39]:
train_feats['paragraph_num_sentences_first'] == train_features['paragraph_num_sentences_first']

KeyError: 'paragraph_num_sentences_first'